In [332]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

In [333]:
def create_year_dataframe(year):
    df = pd.read_csv("survey_results_public_" + str(year) + ".csv", low_memory=False)
    return df

In [334]:
years_data = [2015, 2016, 2017, 2018, 2019]

In [335]:
def create_all_dataframes(all_years):
    dict_df = {}
    for year in years_data:
        dict_df[year] = create_year_dataframe(year)
    return dict_df

In [336]:
all_years = create_all_dataframes(years_data)

In [337]:
all_years[2015].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Select all that apply,Unnamed: 9,...,Unnamed: 212,Unnamed: 213,Select all that apply.7,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221
0,Country,Age,Gender,Tabs or Spaces,Years IT / Programming Experience,Occupation,Desktop Operating System,Desktop Operating System: write-in,Current Lang & Tech: Android,Current Lang & Tech: Arduino,...,Why use Stack Overflow: I don't use Stack Over...,How often are Stack Overflow's answers helpful,Why answer: Help a programmer in need,Why answer: Help future programmers,Why answer: Demonstrate expertise,Why answer: Self promotion,Why answer: Sense of responsibility to developers,Why answer: No idea,Why answer: I don't answer and I don't want to,Why answer: I don't answer but I want to
1,Croatia,25-29,Male,Tabs,2 - 5 years,Back-end web developer,Ubuntu,NaN,NaN,NaN,...,NaN,Usually,NaN,NaN,It feels good to demonstrate my expertise.,Demonstrating my expertise will benefit me,I feel a sense of responsibility to the develo...,NaN,NaN,NaN
2,France,20-24,Male,Spaces,1 - 2 years,Back-end web developer,Windows 7,NaN,NaN,NaN,...,NaN,Usually,NaN,My answer will help lots of people who have th...,It feels good to demonstrate my expertise.,NaN,NaN,NaN,NaN,NaN
3,India,20-24,Male,Tabs,1 - 2 years,Back-end web developer,Windows 7,NaN,NaN,NaN,...,NaN,Rarely,NaN,NaN,NaN,Demonstrating my expertise will benefit me,NaN,NaN,NaN,NaN
4,Latvia,25-29,Male,It depends,6 - 10 years,Back-end web developer,Ubuntu,NaN,NaN,NaN,...,NaN,Usually,It feels good to help a programmer in need,My answer will help lots of people who have th...,It feels good to demonstrate my expertise.,Demonstrating my expertise will benefit me,I feel a sense of responsibility to the develo...,NaN,NaN,NaN


In [338]:
#cleaning 2015 headers
columns_2015 = all_years[2015].iloc[0]
all_years[2015] = all_years[2015][1:]
all_years[2015].columns = columns_2015

<H2> Question 1: How has the survey changed over the year <h2>

In [339]:
def get_yearly_shape(dictionary, year):
    return dictionary[year].shape[0], dictionary[year].shape[1]

In [340]:
get_yearly_shape(all_years, 2019)

(88883, 85)

<H2> Question 2: How have the most common occupations changed in the United States over the last 3 years <h2>

In [341]:
def create_value_df(df, year, column, column_name):
    new_df = pd.DataFrame(df[year][column].value_counts()).reset_index()
    new_df.columns=[column_name, "Count"]
    new_df["Percentage"] = new_df["Count"]/sum(new_df["Count"])
    new_df = new_df.replace("(?i).*mobile.*", "Mobile", regex=True)
    new_df = new_df.groupby("DeveloperType", as_index=False).sum()
    new_df["Year"] = year
    new_df.sort_values("Percentage", ascending=False, inplace=True)
    return new_df

In [342]:
def split_words(df, year, column, column_name):
    all_the_words = defaultdict(int) 
    for row in df[year][column]:
        if isinstance(row, float):
            all_the_words[row] += 1
        else:
            for word in row.split(";"):
                all_the_words[word.strip()] += 1
    new_df = pd.DataFrame(pd.Series(all_the_words)).reset_index()
    new_df.columns = [column_name, "Count"]
    new_df["Percentage"] = new_df["Count"] / sum(new_df["Count"])
    new_df["Year"] = year
    new_df.sort_values('Percentage', ascending=False, inplace=True)
    return new_df

In [343]:
occupation_2015 = create_value_df(all_years, 2015, "Occupation", "DeveloperType")
occupation_2016 = create_value_df(all_years, 2016, "occupation", "DeveloperType")
occupation_2017 = split_words(all_years, 2017, "DeveloperType", "DeveloperType")
occupation_2018 = split_words(all_years, 2018, "DevType", "DeveloperType")
occupation_2019 = split_words(all_years, 2019, "DevType", "DeveloperType")

In [344]:
occupation_df = occupation_2015.append(occupation_2016).append(occupation_2017).append(occupation_2018).append(occupation_2019)

In [345]:
def standardize_developer_types(df, regexes, words):
    new_df = df.replace(regexes, words, regex=True)
    return new_df

In [347]:
regexes = ["(?i).*full-stack.*", "(?i).*back-end.*", "(?i).*front-end.*", "(?i).*embedded.*", "(?i).*desktop.*", "(?i).*executive.*", "(?i).*devops.*", "(?i).*mobile.*" ] 

In [348]:
words = ["Full-stack", "Back-end", "Front-end", "Embedded", "Desktop", "Executive", "DevOps", "Mobile"]

In [349]:
standardized_data_df = standardize_developer_types(occupation_df, regexes, words)

In [363]:
standardized_data_df.groupby(["Year", "DeveloperType"]).mean().sort_values(by="Percentage", ascending=False).to_csv("test.csv")